In [1]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
# @assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
pkgs = [
    "Revise",
    "FASTX",
    "BioSequences",
    "Kmers"
]
# Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
import Mycelia

[ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
[ Info: Skipping precompilation since __precompile__(false). Importing Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd].


KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

KeyError: KeyError: key "usage_request" not found

In [ ]:
PROJECT_BASEDIR = dirname(pwd())
data_dir = joinpath(PROJECT_BASEDIR, "data")
genome_dir = mkpath(joinpath(data_dir, "genomes"))

In [ ]:
reference_fastas = sort(filter(x -> occursin(r"\.fna$", x) && !occursin("normalized", x), readdir(genome_dir, join=true)), by=x->filesize(x))

In [ ]:
reference_fasta = first(reference_fastas)

In [ ]:
filter(x -> occursin(reference_fasta, x) && occursin("10x", x) && occursin(r"\.fq\.gz$", x), readdir(genome_dir, join=true))

In [ ]:
fastq = filter(x -> occursin(reference_fasta, x) && occursin("10x", x) && occursin("filtlong", x) && occursin(r"\.fq\.gz$", x), readdir(genome_dir, join=true))

In [ ]:
initial_k = Mycelia.assess_dnamer_saturation(fastq, plot=false)

In [ ]:
initial_kmer_type = Kmers.DNAKmer{initial_k}

In [ ]:
kmer_iterator = Kmers.EveryKmer{initial_kmer_type}(BioSequences.LongDNA{2}(FASTX.sequence(first(collect(Mycelia.open_fastx(first(fastq)))))))

In [ ]:
graph = Mycelia.fastx_to_kmer_graph(initial_kmer_type, fastq)